先準備好要做RAG資料，.pdf .txt .docx都可以！

找好之後先執行以下程式，去建立資料夾

### 1. 建立資料夾

In [ ]:
import os
upload_dir = "uploaded_docs"
os.makedirs(upload_dir, exist_ok=True)
print(f"請將你的 .txt, .pdf, .docx 檔案放到這個資料夾中： {upload_dir}")

建立完後，請手動上傳該檔案到資料夾中

切記一定要丟進去新建立的資料夾中，之後才讀得到以下設置好的路徑！

### 2. 更新必要套件並引入

In [ ]:
!pip install -U langchain langchain-community pypdf python-docx sentence-transformers faiss-cpu

In [ ]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader, UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

### 3. 依 e5 建議加入

自訂支援 E5 的 embedding 模型（加上 "passage:" / "query:" 前綴）

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

### 4. 載入文件

In [ ]:
folder_path = upload_dir
documents = []
for file in os.listdir(folder_path):
    path = os.path.join(folder_path, file)
    if file.endswith(".txt"):
        loader = TextLoader(path)
    elif file.endswith(".pdf"):
        loader = PyPDFLoader(path)
    elif file.endswith(".docx"):
        loader = UnstructuredWordDocumentLoader(path)
    else:
        continue
    documents.extend(loader.load())

In [ ]:
print(documents)

### 5. 建立向量資料庫

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = splitter.split_documents(documents)

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
vectorstore = FAISS.from_documents(split_docs, embedding_model)

### 6. 儲存向量資料庫

In [ ]:
vectorstore.save_local("faiss_db")

In [ ]:
!zip -r faiss_db.zip faiss_db

In [ ]:
print("✅ 壓縮好的向量資料庫已儲存為 'faiss_db.zip'，請下載此檔案備份。")

再把這個檔案先丟到Drive上，並設成公開，複製其URL

再使用 [Google Drive Direct Link Generator](IwZXh0bgNhZW0CMTAAAR5ji3w5J7kt2m2qRX7KXWKUKrxKVIj5nqQP5xKQb7fa2hHp97uh2DfpVmO07Q_aem_QX9JsZTBwEnvmiXwQtpC0g) 將剛剛的URL 進行轉換



 資料集：1 0 2 年 度 公 民 營 汽 車 駕 駛 人
訓 練 機 構 小 型 車 駕 駛 教 練 訓 練 交通法規

### 解壓縮向量資料庫

In [ ]:
URL = "https://drive.google.com/uc?export=download&id=1RtzdqVpoNbzSI0EnE-OZNxBjK3dEVtYE"

!wget -O faiss_db.zip "$URL"

In [ ]:
!unzip faiss_db.zip

In [ ]:
!pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [ ]:
from openai import OpenAI
import gradio as gr

In [ ]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

載入 faiss_db

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

設定好我們要的 LLM

In [ ]:
import os
from google.colab import userdata

金鑰那邊設定完後

In [ ]:
api_key = userdata.get('Gemini')

In [ ]:
# 先測試一下 Gemini 的使用

from google import genai

client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Hello"
)
print(response.text)

In [ ]:
system_prompt = "你是台灣交通法規的專家，請全程使用繁體中文回答問題"

prompt_template = """
根據下列資料回答問題：
{retrieved_chunks}

使用者的問題是：{question}

請根據資料內容回覆，若資料不足請告訴同學可以請他打電話去諮詢交通部
"""

### 使用 RAG 來回應

搜尋與使用者問題相關的資訊，根據我們的 prompt 樣版去讓 LLM 回應。

In [ ]:
chat_history = []

def chat_with_rag(user_input):
    global chat_history

    # 🔍 1. 從向量資料庫取回相關文件
    docs = retriever.get_relevant_documents(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    # 🧠 2. 自訂 Prompt（你需要先定義 prompt_template 和 system_prompt）
    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    combined_prompt = f"{system_prompt}\n\n{final_prompt}"

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[
            {
                "role": "user",
                "parts": [{"text": combined_prompt}]
            }
        ]
    )

    if response.candidates and response.candidates[0].content.parts:
        answer = response.candidates[0].content.parts[0].text
    else:
        return "No valid response received from the API."

    chat_history.append((user_input, answer))
    print("Chat history:", chat_history)

    return answer


### 用 Gradio 打造 Web App

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# 🎓 AI 交通法規大王")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="請輸入你的問題...")

    def respond(message, chat_history_local):
        response = chat_with_rag(message)
        chat_history_local.append((message, response))
        return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)